In [8]:
import plotly.offline as py
import plotly.graph_objs as go
import pandas as pd

py.offline.init_notebook_mode(connected=True)

#import data

#datapath = "C:\Users\Dan Steffy\Dropbox\Research\VRPVIS\\"
datapath = "/Users/TomCreo/Desktop/"

df_ArrivalTimes = pd.read_excel(str(datapath+"Assignment30.xlsx"), sheet_name="ArrivalTimes")
df_Locations = pd.read_excel(str(datapath+"Assignment30.xlsx"), sheet_name="Locations")
df_ArrivalLoad = pd.read_excel(str(datapath+"Assignment30.xlsx"), sheet_name="ArrivalLoad")
df_Demand = pd.read_excel(str(datapath+"Assignment30.xlsx"), sheet_name="Demand")
df_Requests = pd.read_csv(str(datapath+"Google_Campus_Data_217_Shifted.csv"))

#to store requestID associated with each stop
VehRequestID = []

#to store latitude associated with each stop
VehLat = []

#to store longitudes associated with each stop
VehLong = []

#marker fill based on pickup or dropoff
VehFill = []

#hovertext RequestID
VehText = []


#create lists of lists
for v in range(30):
    VehRequestID.append([])
    VehLat.append([])
    VehLong.append([])
    VehFill.append([])
    VehText.append([])
    
#calculate request ID associated with each stop
#extract latitude/longitude from requestID
#assign marker fill: pickup = 'circle', dropoff = 'circle-open'

clist = df_Locations.columns
for v in range(30):
    for i, location in enumerate(df_Locations[clist[v]].dropna(),start=1):
        vehdemand =  df_Demand[clist[v]].dropna()[i]
        if vehdemand == 1:
            VehRequestID[v].append(location/2)
            VehLat[v].append(df_Requests.fromLatitude[location/2])
            VehLong[v].append(df_Requests.fromLongitude[location/2])
            VehFill[v].append('circle')
            VehText[v].append('Pickup RequestID '+str(int(location/2)))
        elif vehdemand == -1:
            VehRequestID[v].append((location-1)/2)
            VehLat[v].append(df_Requests.toLatitude[(location-1)/2])
            VehLong[v].append(df_Requests.toLongitude[(location-1)/2])
            VehFill[v].append('circle-open')
            VehText[v].append('Dropoff RequestID '+str(int((location-1)/2)))

    
data = []
for v in range(30):
    trace = go.Scatter3d(
        x=VehLong[v],
        y=VehLat[v],
        z=df_ArrivalTimes[clist[v]].dropna(),
        name='Vehicle' + str(v),
        mode='lines+markers',
        marker=dict(
            size=6,
            symbol=VehFill[v],
            line=dict(
                width=0.5
            ),
            opacity=0.7
        ),
        line=dict(),
        text=VehText[v]
    )
    data.append(trace)

layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    ),
    legend=dict(
        x=0,
        y=0,
        traceorder='normal',
        font=dict(
            family='sans-serif',
            size=12,
            color='#000'
        ),
        bgcolor='#E2E2E2',
        bordercolor='#FFFFFF',
        borderwidth=2
    ),
    title="Route Assignments for Initial Solution (30 Vehicles)",
    scene = dict(
        xaxis = dict(
            title = 'Location Longitude',
            titlefont=dict(
                family='Arial, sans-serif',
                size=12
            ),
            tickfont=dict(
                family='Arial, sans-serif',
                size=10
            ),
        ),
        yaxis = dict(
            title = 'Location Latitude',
            titlefont=dict(
                family='Arial, sans-serif',
                size=12
            ),
            tickfont=dict(
                family='Arial, sans-serif',
                size=10
            )
        ),
        zaxis = dict(
            title = 'Arrival Time (seconds after midnight)',
            titlefont=dict(
                family='Arial, sans-serif',
                size=12
            ),
            tickfont=dict(
                family='Arial, sans-serif',
                size=10
            )
        )
    )
)

fig = go.Figure(data=data, layout=layout)
py.offline.iplot(fig)
